In [1]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home'

In [2]:
! echo $JAVA_HOME

/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home


In [3]:
import pyspark as ps
from pyspark import SparkContext
from pyspark.ml import PipelineModel
from pyspark.sql import functions as f
from pyspark.sql import types as t
import numpy as np
import pandas as pd

In [4]:
# sc = ps.SparkContext(appName = 'test2app')

In [5]:
# create a SparkContext
# load saved pipeline model from the folder 'model'
sc = ps.SparkContext()
sqlContext = ps.sql.SQLContext(sc)
loadedModel = PipelineModel.load("/Users/mineryang/Desktop/PipelineIrisModel2")

In [6]:
schema = t.StructType(
    [
        t.StructField("SepalLengthCm",t.DoubleType()),
        t.StructField("SepalWidthCm",t.DoubleType()),
        t.StructField("PetalLengthCm",t.DoubleType()),
        t.StructField("PetalWidthCm",t.DoubleType()),
    ]
)

In [7]:
user_input = {
    'SepalLengthCm':5.2,
    'SepalWidthCm':2.7,
    'PetalLengthCm':3.9,
    'PetalWidthCm':1.4
}

testdf=sqlContext.createDataFrame([user_input],schema)

In [8]:
prediction = loadedModel.transform(testdf)
prediction_label = prediction.select(prediction['predictedLabel']).collect()
res = [r['predictedLabel'] for r in prediction_label][0]
print(res)

Iris-versicolor


In [9]:
from flask import Flask
from flask import request, redirect, url_for
from flask_cors import CORS
from flask_restful import reqparse, abort, Api, Resource

# create a Flask instance
app = Flask(__name__)
CORS(app)
api = Api(app)

In [10]:
# create a parser
# fill a parser with information about arguments 
parser = reqparse.RequestParser()
parser.add_argument("sepal_length",type=float)
parser.add_argument("sepal_width",type=float)
parser.add_argument("petal_length",type=float)
parser.add_argument("petal_width",type=float)

In [11]:
@app.route('/')
def index():
    return "welcome"

@app.route('/predict',methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        args = parser.parse_args()
        sl = args["sepal_length"]
        sw = args["sepal_width"]
        pl = args["petal_length"]
        pw = args["petal_width"]
        
        X = {
            'SepalLengthCm':sl,
            'SepalWidthCm':sw,
            'PetalLengthCm':pl,
            'PetalWidthCm':pw
        }
        print(args['sepal_length'])
        df=sqlContext.createDataFrame([X],schema)
        prediction = loadedModel.transform(df)
        prediction_label = prediction.select(prediction['predictedLabel']).collect()
        res = [r['predictedLabel'] for r in prediction_label][0]
        print(res)
        return res
    else:
        return "in get situation"

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


3.5


127.0.0.1 - - [08/Apr/2020 15:41:58] "POST /predict HTTP/1.1" 200 -


Iris-virginica


127.0.0.1 - - [08/Apr/2020 15:42:08] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 15:42:08] "POST /predict HTTP/1.1" 200 -


1.0
Iris-setosa
